In [23]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random
import time
import json
import os
from threading import Thread
import multiprocessing as mp



from networkx.algorithms import community
from matplotlib.pyplot import figure

# my functions implemented in ./src/
from vkprint import vkprint
from crawling_algorythms import Crawler_RC,Crawler_RW,Crawler_DFS,Crawler_BFS,Crawler_MOD,Crawler_MED, METRICS_LIST
from utils import import_graph,draw_graph,get_percentile,draw_percentile_heatmap,draw_nodes_history,draw_scores_history

ImportError: cannot import name 'draw_nodes_history' from 'utils' (/home/denisaivazov/crawling/src/utils.py)

In [2]:
#!pip install Queue
import queue as queue #import queue
def treading_crawler(seed_num,big_graph,crawling_method,node_seed,b,percentile_set):
    global q
    print('thread alive',seed_num,q.qsize())
    thread_crawler=crawling_method(big_graph, node_seed = node_seed, budget = b,percentile_set = percentile_set)
    
    counter = 0
    while counter<b:
        if counter %1000 == 1:
            print(counter,seed_num)
        counter+=1
        thread_crawler.sampling_process()
        #if counter %100:
            #print(counter)
    #print(seed_num,thread_crawler.observed_history)
    #return_dict[seed_num] = thread_crawler.observed_history
    vkprint('thread ready',seed_num, q.qsize())
    q.put(thread_crawler.observed_history)
    print(seed_num,'putted')

In [15]:
%%time
## макропараметры всей задачи
graph_name = 'gnutella'
methods = {'RW':Crawler_RW,'RC':Crawler_RC,'DFS':Crawler_DFS, 'BFS':Crawler_BFS, 
              'MOD':Crawler_MOD,'MED':Crawler_MED} #'AFD',
b = 100
seed_count =8
top_percetile = 10

# общие множества и вспомогательное, лучше не менять
method_color = {'AFD':'pink','RC':'grey','RW':'green','DFS':'black', 'BFS':'blue', 'MED':'cyan','MOD':'red'}
graph_name_list = ['importing','wikivote','hamsterster','DCAM','gnutella',  'dblp2010',  'github']  #'slashdot',
linestyles = [':', '--', '-.','--']*seed_count


# Дальше лучше не трогать
t00 = time.time()
big_graph = import_graph(graph_name)
b = min(big_graph.number_of_nodes(),b)
percentile, percentile_set = get_percentile(big_graph,graph_name,top_percetile) # берём топ 10 процентов вершин
nx.write_gml(big_graph, "../data/Graphs/"+ graph_name +'_BIG.graph')
vkprint("big_graph "+graph_name+"\nb=",b,"seed count=",seed_count, "methods:",methods, "\n percentile",percentile)


#draw_graph(big_graph,graph_name) # отрисовываем граф, если не очень большой
seeds = random.sample(set(big_graph.nodes),seed_count) # список начальных вершин, по которым мы будем проходиться
crawler = dict()   
crawler_avg = dict()

seeds = random.sample(set(big_graph.nodes),seed_count)
history = dict((method, []*seed_count) for method in methods)
crawler = dict((method, {'nodes':[],'degrees':[],'k_cores':[],'eccentricity':[],
                                  'betweenness_centrality':[]}) for method in methods)
crawler_avg = dict((method, {'nodes':[],#np.zeros([len(methods)*seed_count,b+1]),
                                  'degrees':[],'k_cores':[],'eccentricity':[],'betweenness_centrality':[]}) for method in methods)



# для каждого метода отдельный список тредов, и всё в виде словаря
threads =dict()

for method in methods:   
    #process_queue = list(seeds)    
    #while len(process_queue)>0:
    crawler_avg[method] = dict({i: np.zeros(b) for i in METRICS_LIST}, **{'nodes': np.zeros(b)})
    property_history_hist = {'nodes':[],'degrees':[],'k_cores':[],'eccentricity':[],'betweenness_centrality':[]}
    
    threads [method] = []
    
    for seed_num in range(seed_count):
        #process_method = process_queue.pop()
        t0 =time.time()
        #algorythm = methods[method]
        
        
    q = mp.Queue()
    for seed_num in range(seed_count):
        threads[method].append(mp.Process(daemon = True,target=treading_crawler, args=(seed_num,big_graph,methods[method],seeds[seed_num],b,percentile_set)))
        #threads[method][seed_num].setDaemon = True
        threads[method][seed_num].start()
        
    for seed_num in range(seed_count):
        history[method].append(q.get())
        
    q.close()
    q.join_thread()    
        
    for seed_num in range(seed_count):
        threads[method][seed_num].join()


        for prop in METRICS_LIST:
            property_history_hist[prop] = np.array([i for i in history[method][seed_num][prop]])
            crawler_avg[method][prop] += property_history_hist[prop]
        
        crawler_avg[method]['nodes'] += np.array(history[method][seed_num]['nodes'])
    vkprint(graph_name, seed_count, method, seed_num,' it took ', round(((time.time() -t0)/60),3),'minutes'  )


 big_graph gnutella
b= 100 seed count= 8 methods: {'RW': <class 'crawling_algorythms.Crawler_RW'>, 'RC': <class 'crawling_algorythms.Crawler_RC'>, 'DFS': <class 'crawling_algorythms.Crawler_DFS'>, 'BFS': <class 'crawling_algorythms.Crawler_BFS'>, 'MOD': <class 'crawling_algorythms.Crawler_MOD'>, 'MED': <class 'crawling_algorythms.Crawler_MED'>} 
 percentile {'degrees': 13, 'k_cores': 5, 'eccentricity': 24, 'betweenness_centrality': 284783}
thread alive 0 0
1 0
thread alive 1 0
1 1
thread alive 2 0
1 2
thread alive 3 0
thread alive 5 0
1 3
thread alive 4 0
thread alive 6 0
1 5
thread alive 7 0
1 4
1 6
1 7
 thread ready 0 0
 thread ready 2 0
 thread ready 1 0
 thread ready 5 0
 thread ready 3 0
 thread ready 4 0
 thread ready 6 0
 thread ready 7 0
0 putted
5 putted
2 putted
1 putted
4 putted
7 putted
6 putted
3 putted
 gnutella 8 RW 7  it took  0.024 minutes
thread alive 0 0
1 0
thread alive 1 0
1 1
thread alive 2 0
1 2
thread alive 3 0
1 3
thread alive 4 0
1 4
thread alive 5 0
1 5
threa

TypeError: draw_nodes_history() missing 2 required positional arguments: 'history' and 'methods'

In [20]:
draw_nodes_history(history,crawler_avg, methods,graph_name,seed_count,method_color)
draw_scores_history(percentile_set,methods,graph_name,seed_count,b)



draw_percentile_heatmap(percentile_set,graph_name,seed_count,b, normalized=True, venn_on=True)




TypeError: draw_nodes_history() takes 2 positional arguments but 6 were given

In [56]:
vkprint('i have done everything!!!!')

 i have done everything!!!!


1918